In [1]:
# pip install imbalanced-learn

from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

2024-07-15 15:46:32.421069: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-15 15:46:32.741770: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 15:46:32.752619: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-15 15:46:32.776264: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 15:46:32.806924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

In [2]:
import os
import numpy as np
import rasterio
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def load_dataset(data_dir, img_size=(256, 256)):
    """
    Load dataset from specified directory.

    Parameters:
    - data_dir (str): Directory containing train, val, and test subdirectories.
    - img_size (tuple): Desired size of the image (height, width).

    Returns:
    - train_data (tuple): Tuple containing (X_train, y_train).
    - val_data (tuple): Tuple containing (X_val, y_val).
    - test_data (tuple): Tuple containing (X_test, y_test).
    """
    train_dir = os.path.join(data_dir, 'train')
    val_dir = os.path.join(data_dir, 'val')
    test_dir = os.path.join(data_dir, 'test')

    # Load training data
    X_train, y_train = load_data_from_dir(os.path.join(train_dir, 'input'), os.path.join(train_dir, 'output'), img_size)
    # Load validation data
    X_val, y_val = load_data_from_dir(os.path.join(val_dir, 'input'), os.path.join(val_dir, 'output'), img_size)
    # Load test data
    X_test, y_test = load_data_from_dir(os.path.join(test_dir, 'input'), os.path.join(test_dir, 'output'), img_size)

    # Preprocess to handle NaN values
    X_train = preprocess_data(X_train)
    X_val = preprocess_data(X_val)
    X_test = preprocess_data(X_test)

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

def preprocess_data(images):
    """
    Preprocesses input images to handle NaN values.

    Parameters:
    - images (numpy.ndarray): Array of input images.

    Returns:
    - images (numpy.ndarray): Processed array of input images.
    """
    # Replace NaN values with 0
    images[np.isnan(images)] = 0

    return images

def load_data_from_dir(input_dir, output_dir, img_size):
    """
    Load data (images and labels) from input and output directories.

    Parameters:
    - input_dir (str): Directory containing input images.
    - output_dir (str): Directory containing output images.
    - img_size (tuple): Desired size of the image (height, width).

    Returns:
    - images (numpy.ndarray): Array of loaded input images.
    - labels (numpy.ndarray): Array of corresponding output images.
    """
    images = []
    labels = []

    for filename in os.listdir(input_dir):
        if filename.endswith('.tif'):
            # Load input image (X)
            input_path = os.path.join(input_dir, filename)
            img = load_tiff_image(input_path, img_size)
            images.append(img)

            # Load corresponding output image (y)
            output_filename = filename.replace('.tif', '_cl.tif')
            output_path = os.path.join(output_dir, output_filename)
            label = load_tiff_image(output_path, img_size, is_label=True)
            label[np.isnan(label)] = 0 
            labels.append(label)

    if images and labels:
        images = np.array(images)
        labels = np.array(labels)

    return images, labels

def load_tiff_image(path, img_size, is_label=False):
    """
    Load a TIFF image from specified path.

    Parameters:
    - path (str): Path to the TIFF image.
    - img_size (tuple): Desired size of the image (height, width).
    - is_label (bool): Whether the image is a label image.

    Returns:
    - img (numpy.ndarray): Loaded image as a numpy array.
    """
    with rasterio.open(path) as src:
        img = src.read()

    # Reshape and resize if necessary
    img = img.transpose(1, 2, 0)  # Change from bands x height x width to height x width x bands
    img = img[:img_size[0], :img_size[1], :]  # Resize to desired size

    if is_label:
        # Convert label image to binary classification (1 for Marine Debris, 0 for others)
        img = (img == 1).astype(np.uint8)  # Assuming Marine Debris class is encoded as 1

    return img

# Example usage:
data_dir = 'MARIDA'
(X_train, y_train), (X_val, y_val), (X_test, y_test) = load_dataset(data_dir)

print(f"Training data: X_train shape = {X_train.shape}, y_train shape = {y_train.shape}")
print(f"Validation data: X_val shape = {X_val.shape}, y_val shape = {y_val.shape}")
print(f"Testing data: X_test shape = {X_test.shape}, y_test shape = {y_test.shape}")


Training data: X_train shape = (694, 256, 256, 11), y_train shape = (694, 256, 256, 1)
Validation data: X_val shape = (328, 256, 256, 11), y_val shape = (328, 256, 256, 1)
Testing data: X_test shape = (359, 256, 256, 11), y_test shape = (359, 256, 256, 1)


In [3]:
print(y_train.shape)

(694, 256, 256, 1)


In [4]:
print(X_train.shape)

(694, 256, 256, 11)


In [5]:
from imblearn.over_sampling import SMOTE
X = np.reshape(X_train, (256*256*694, 11))
Y = np.reshape(y_train, (256*256*694, 1))
print(Y.shape)

(45481984, 1)


In [6]:
sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X, Y)
print(X_res.shape)
print(y_res.shape)

(90960082, 11)
(90960082,)


Band 1 - Coastal Aerosol: 443.9 nm
Band 2 - Blue: 496.6 nm
Band 3 - Green: 560.0 nm
Band 4 - Red: 664.5 nm
Band 5 - Vegetation Red Edge: 703.9 nm
Band 6 - Vegetation Red Edge: 740.2 nm
Band 7 - Vegetation Red Edge: 782.5 nm
Band 8 - NIR (Near Infrared): 835.1 nm
Band 8A - Narrow NIR: 864.8 nm
Band 9 - Water Vapor: 945.0 nm
Band 10 - SWIR - Cirrus: 1373.5 nm
Band 11 - SWIR: 1613.7 nm
Band 12 - SWIR: 2202.4 nm

In [7]:
import pandas as pd
Coastal = pd.DataFrame(X_res).iloc[:, 0]  # costel 
Blue = pd.DataFrame(X_res).iloc[:, 1] # blue 
Green = pd.DataFrame(X_res).iloc[:, 2] # green 
RED = pd.DataFrame(X_res).iloc[:, 3] #
RedEdge1 = pd.DataFrame(X_res).iloc[:, 4]
RedEdge2 = pd.DataFrame(X_res).iloc[:, 5]
RedEdge3 = pd.DataFrame(X_res).iloc[:, 6]
NIR = pd.DataFrame(X_res).iloc[:, 7]
Narrow_NIR = pd.DataFrame(X_res).iloc[:, 8]
SWIR1 = pd.DataFrame(X_res).iloc[:, 9]
SWIR2 = pd.DataFrame(X_res).iloc[:, 10]

# print(first_column.tolist())


In [8]:
Blue.shape

(90960082,)

# NDVI 

In [9]:
NDVI = (NIR - RED) / (NIR + RED)
print(np.isinf(NDVI).sum())
print(np.isinf(NDVI).sum())

0
0


# Plastic Index

In [10]:
Plastic = NIR / (NIR - RED)
print(np.isinf(Plastic).sum())
print(np.isinf(-Plastic).sum())

23
23


# Floating Littere Index (FDI)

In [11]:
FDI = NIR - (RedEdge2+ (SWIR1 - RedEdge2) * ((842 - 665)/(1610 - 665)) * 10  )
np.isinf(FDI).sum()
np.isinf(-FDI).sum()

0

# Water Index

In [12]:
WaterIndex = (Green-NIR)/ (Green+NIR)
np.isinf(WaterIndex).sum()
np.isinf(-WaterIndex).sum()

0

In [13]:
y_res= pd.DataFrame(y_res)

In [14]:
X_train = pd.concat([FDI,WaterIndex,Plastic,NDVI,SWIR1,NIR,RED,y_res],axis=1)
X_train.shape

(90960082, 8)

In [15]:
np.isinf(X_train).sum()

0     0
1     0
2    23
3     0
9     0
7     0
3     0
0     0
dtype: int64

In [16]:
mask = (X_train == np.inf).any(axis=1) | (X_train == -np.inf).any(axis=1)
data_filtered = X_train[~mask]

In [17]:
data_filtered.shape

(90960059, 8)

In [18]:
data_filtered

,0,1,2,3,9,7,3,0
0,0.022989,0.068216,-14.589310,-0.033136,0.046157,0.057946,0.061918,0
1,0.021560,0.075667,-12.986708,-0.037074,0.046157,0.056517,0.060869,0
2,0.022126,0.063699,-13.046866,-0.036909,0.046458,0.056925,0.061288,0
3,0.021820,0.073105,-17.124535,-0.028370,0.046458,0.056619,0.059926,0
4,0.019559,0.081620,-10.930187,-0.043744,0.046758,0.055191,0.060240,0
...,...,...,...,...,...,...,...,...
90960077,0.158873,-0.106069,8.628856,0.061509,0.054773,0.140608,0.124313,1
90960078,0.095715,-0.106782,7.534398,0.071079,0.040862,0.103614,0.089862,1
90960079,0.039913,0.060976,-7.425355,-0.063089,0.029870,0.045770,0.051934,1
90960080,0.049998,0.063030,-80.625679,-0.006163,0.024352,0.053888,0.054556,1


In [19]:
np.isinf(data_filtered).sum()

0    0
1    0
2    0
3    0
9    0
7    0
3    0
0    0
dtype: int64

In [20]:
X = data_filtered.iloc[:-1, :-1]  # Features (all columns except the last one)
Y = data_filtered.iloc[:-1,-1]    # Target variable (last row, last column)
print(X.shape, Y.shape)

(90960058, 7) (90960058,)


In [21]:
Y.unique()

array([0, 1], dtype=uint8)

# TEST

In [22]:
test_y = np.reshape(y_test,(359*256*256,1))
test_x = np.reshape(X_test,(359*256*256,11))
test_y.shape

(23527424, 1)

In [23]:
test_y = np.reshape(y_test,(359*256*256,1))
test_x = np.reshape(X_test,(359*256*256,11))
test_y.shape

(23527424, 1)

In [24]:
Coastal = pd.DataFrame(test_x).iloc[:, 0]  # costel 
Blue = pd.DataFrame(test_x).iloc[:, 1] # blue 
Green = pd.DataFrame(test_x).iloc[:, 2] # green 
RED = pd.DataFrame(test_x).iloc[:, 3] #
RedEdge1 = pd.DataFrame(test_x).iloc[:, 4]
RedEdge2 = pd.DataFrame(test_x).iloc[:, 5]
RedEdge3 = pd.DataFrame(test_x).iloc[:, 6]
NIR = pd.DataFrame(test_x).iloc[:, 7]
Narrow_NIR = pd.DataFrame(test_x).iloc[:, 8]
SWIR1 = pd.DataFrame(test_x).iloc[:, 9]
SWIR2 = pd.DataFrame(test_x).iloc[:, 10]

# print(first_column.tolist())


In [25]:
NDVI = (NIR - RED) / (NIR + RED)
print(np.isinf(NDVI).sum())
print(np.isinf(NDVI).sum())

0
0


In [26]:
Plastic = NIR / (NIR - RED)
print(np.isinf(Plastic).sum())
print(np.isinf(-Plastic).sum())

0
0


In [27]:
FDI = NIR - (RedEdge2+ (SWIR1 - RedEdge2) * ((842 - 665)/(1610 - 665)) * 10  )
np.isinf(FDI).sum()
np.isinf(-FDI).sum()

0

In [28]:
WaterIndex = (Green-NIR)/ (Green+NIR)
np.isinf(WaterIndex).sum()
np.isinf(-WaterIndex).sum()

0

In [29]:
X_test = pd.concat([FDI,WaterIndex,Plastic,NDVI,SWIR1,NIR,RED],axis=1)
X_test.shape

(23527424, 7)

In [30]:
np.isinf(X_test).sum()

0    0
1    0
2    0
3    0
9    0
7    0
3    0
dtype: int64

In [31]:
X_test

,0,1,2,3,9,7,3
0,0.008804,0.389591,-1.891992,-0.209031,0.007796,0.011799,0.018035
1,0.008497,0.422405,-1.979375,-0.201664,0.007796,0.011492,0.017298
2,0.010325,0.385112,-2.144156,-0.189096,0.007896,0.012515,0.018351
3,0.009610,0.425812,-1.800797,-0.217316,0.007896,0.011799,0.018351
4,0.010208,0.410518,-2.078451,-0.193915,0.007596,0.012106,0.017930
...,...,...,...,...,...,...,...
23527419,0.026898,0.646449,-0.400212,-0.555425,0.001669,0.015145,0.052989
23527420,0.026775,0.672550,-0.365080,-0.577981,0.001368,0.013917,0.052039
23527421,0.026263,0.680150,-0.345114,-0.591636,0.001368,0.013406,0.052250
23527422,0.026463,0.651327,-0.396822,-0.557524,0.001869,0.014634,0.051511


# SVM

In [ ]:
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Example data generation (replace with your own data)
X, y = make_classification(n_samples=90960058, n_features=7, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train SVM model with parallel processing
svm_model = SVC(kernel='linear', C=1.0, random_state=42)
svm_model.fit(X, Y)


In [ ]:

# Predict on test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(test_y, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
import joblib

# Save the model to disk
filename = 'svm_model.sav'
joblib.dump(svm_model, filename)

print(f"Saved SVM model to {filename}")

# Random Forest

In [32]:
# Initialize and fit the RandomForestClassifier with specified parameters
rf_classifier = RandomForestClassifier(n_estimators=500, max_depth=11, random_state=42, n_jobs=6)
rf_classifier.fit(X, Y)

RandomForestClassifier(max_depth=11, n_estimators=500, n_jobs=6,
                       random_state=42)

In [33]:
result = rf_classifier.predict(X_test)

In [35]:
from sklearn.metrics import classification_report

In [36]:
report = classification_report(result, test_y)
report

'              precision    recall  f1-score   support\n\n           0       0.99      1.00      1.00  23407653\n           1       0.90      0.00      0.01    119771\n\n    accuracy                           0.99  23527424\n   macro avg       0.95      0.50      0.50  23527424\nweighted avg       0.99      0.99      0.99  23527424\n'

In [37]:
print(report)

              precision    recall  f1-score   support

           0       0.99      1.00      1.00  23407653
           1       0.90      0.00      0.01    119771

    accuracy                           0.99  23527424
   macro avg       0.95      0.50      0.50  23527424
weighted avg       0.99      0.99      0.99  23527424



In [38]:
def write_to_file(filename, content):
    with open(filename, 'w') as f:
        f.write(content)
# Example usage:
filename = '1000_decision_tree.txt'

write_to_file(filename, report)

In [39]:
import joblib

joblib.dump(rf_classifier, "random_forest_1000_decision_tree.joblib")

['random_forest_1000_decision_tree.joblib']

In [ ]:
import os

def shutdown():
    if os.name == 'posix':  # For UNIX/Linux/MacOS
        os.system('shutdown -h now')
    elif os.name == 'nt':  # For Windows
        os.system('shutdown /s /t 1')
    else:
        raise OSError(f"Unsupported operating system: {os.name}")

# Calling the shutdown function
shutdown()


In [ ]:
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Define ImageDataGenerator with rotation augmentation
# datagen = ImageDataGenerator(
#     rotation_range=45,  # Rotate images randomly up to 45 degrees
#     rescale=1./255  # Normalize pixel values (assuming pixel range 0-255)
# )

# # Example usage:
# batch_size = 32
# # Create generators for training and validation data
# train_generator = datagen.flow(X_train, y_train, batch_size=batch_size)
# val_generator = datagen.flow(X_val, y_val, batch_size=batch_size)

# # Note: No need to augment validation data, so we only apply rotation augmentation to training data


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.losses import SparseCategoricalCrossentropy

# # Compute class weights based on frequency
# def compute_class_weights(y_train):
#     class_weights = {}
#     total_samples = len(y_train)
#     unique_classes = np.unique(y_train)
#     class_counts = np.bincount(y_train.flatten())
    
#     for i, count in enumerate(class_counts):
#         class_weights[i] = (1 / count) * (total_samples / len(unique_classes))
    
#     return class_weights

# # Example usage
# # Assuming y_train is your training labels (shape: (694, 256, 256))
# y_train_flat = y_train.flatten()
# class_weights = compute_class_weights(y_train_flat)

# # Define weighted loss function
# loss_function = SparseCategoricalCrossentropy(from_logits=True, weight=class_weights)

# # Compile your model with this loss function
# model.compile(optimizer='adam', loss=loss_function, metrics=['accuracy'])

# # Train your model using the generators
# history = model.fit(train_generator, epochs=num_epochs, validation_data=val_generator)
